# Introduction to Document Similarity with Elasticsearch


## Document Distance
Short intro to document similarity - finding a way to represent the distance between documents. 

Two things needed:
 - The first is to encode the documents as vectors. There are options for this (one-hot, frequency, distributed). 
 - The second is to decide how to measure distance - Euclidean? Manhattan? Cosine? There are tradeoffs but cosine is a good place to start with text. Why? Documents encoded as vectors are sparse; each vector could be as long as the number of unique words across the full corpus.  That means that two documents of very different lengths (e.g. a single recipe and a cookbook), could be encoded with the same length vector, which might overemphasize the magnitude of the book's document vector at the expense of the recipe's document vector. Cosine distance helps to correct for variations in vector magnitudes resulting from uneven length documents, and enables us to measure the distance between the book and recipe.
 
Document similarity is something we cover in the book - in chapter 6 and then in chapter 10 where we build a chatbot that, among other things, uses a nearest neigbor search to recommend recipes that are similar to the ingredients listed by the user. One of my observations during the prototyping phase for that chapter is how slow vanilla nearest neighbor search is. This led me to think about different ways to optimize the search, from using variations like k-ball, to using other Python libraries like Spotify's Annoy, and also to other kind of tools altogether that attempt to deliver a similar results as quickly as possible. Like ElasticSearch!
 
## What is Elasticsearch
Elasticsearch is a text search engine that leverages the information retrieval library Lucene together with a key-value store to ...


## Get the Sample Corpus
The Yellowbrick machine learning visualization library hosts several datasets wrangled from the UCI Machine Learning Repository or built by District Data Labs to present the examples used throughout this documentation, one of which is a text corpus of news documents collected from different domain area RSS feeds. If you haven't downloaded the data, you can do so by running:

    $ python -m yellowbrick.download
    
This should create a folder named data in your current working directory that contains all of the datasets. You can load a specified dataset as follows:


In [1]:
import os

from sklearn.datasets.base import Bunch
from yellowbrick.download import download_all

## The path to the test data sets
FIXTURES  = os.path.join(os.getcwd(), "data")

## Dataset loading mechanisms
datasets = {
    "hobbies": os.path.join(FIXTURES, "hobbies")
}


def load_data(name, download=True):
    """
    Loads and wrangles the passed in text corpus by name.
    If download is specified, this method will download any missing files. 
    """
    
    # Get the path from the datasets 
    path = datasets[name]
    
    # Check if the data exists, otherwise download or raise 
    if not os.path.exists(path):
        if download:
            download_all() 
        else:
            raise ValueError((
                "'{}' dataset has not been downloaded, "
                "use the download.py module to fetch datasets"
            ).format(name))
    
    # Read the directories in the directory as the categories. 
    categories = [
        cat for cat in os.listdir(path) 
        if os.path.isdir(os.path.join(path, cat))
    ]
    
    
    files  = [] # holds the file names relative to the root 
    data   = [] # holds the text read from the file 
    target = [] # holds the string of the category 
        
    # Load the data from the files in the corpus 
    for cat in categories:
        for name in os.listdir(os.path.join(path, cat)):
            files.append(os.path.join(path, cat, name))
            target.append(cat)
            
            with open(os.path.join(path, cat, name), 'r') as f:
                data.append(f.read())
        
    
    # Return the data bunch for use similar to the newsgroups example
    return Bunch(
        categories=categories,
        files=files,
        data=data,
        target=target,
    )

In [2]:
corpus = load_data('hobbies')

hobby_types  = {}

for category in corpus.categories:
    texts = []
    for idx in range(len(corpus.data)):
        if corpus['target'][idx] == category:
            texts.append(corpus.data[idx])
    hobby_types[category] = texts
    
food_stories = [text for text in hobby_types['cooking']]

In [3]:
print(food_stories[5])


	   Smoothies are one of our favorite breakfast options year-round, but especially in the winter because it's an easy way to brighten the mornings by adding extra fruit to our diets. And since frozen fruit works just as well (if not better — hello, icy, creamy texture), there's no worry about whether it's in season.



	   If you're stuck in the doldrums of gray days, try these tropical smoothies. It won't be quite like you're relaxing on a white-sand beach, but it'll be closer to it than your normal bowl of oatmeal.



	         Sometimes, you just have to go with a classic. This smoothie recipe is one of our standbys. We might dress it up or down with a touch of mint or swap in coconut water, but we can always count on it to be delicious.



	         You'll definitely get bonus points if you use a fresh coconut to make this smoothie. But because we know mornings are hectic, you can also swap in a can of coconut milk, and it will be just as creamy and delicious.



	        Adding m

## Start Elasticsearch
In the command line, start elasticsearch.

```bash
$ cd elasticsearch-<version>        
$ ./bin/elasticsearch
```

## Create an Index
Now we will create an index. Think of an index as a database in PostgreSQL or MongoDB. An Elasticsearch cluster can contain multiple _indices_ (e.g. relational or noSql databases), which in turn contain multiple _types_ (similar to MongoDB collections or PostgreSQL tables). These types hold multiple _documents_ (similart o MongoDB documents or PostgreSQL rows), and each document has _properties_ (like MongoDB document key-values or PostgreSQL columns).

```bash
curl -X PUT "localhost:9200/cooking " -H 'Content-Type: application/json' -d'
{
    "settings" : {
        "index" : {
            "number_of_shards" : 1,
            "number_of_replicas" : 1
        }
    }
}
'
```

 And the response:
 
```bash
{"acknowledged":true,"shards_acknowledged":true,"index":"cooking"}
```

### Get All Indices or Delete a Specific Index

```bash
$ curl -X GET "localhost:9200/_cat/indices?v"
```

```bash
$ curl -X DELETE "localhost:9200/cooking"
```

### Manually Add Some Documents to the Index

```bash
$ curl -X PUT "localhost:9200/cooking/_doc/1?pretty" -H 'Content-Type: application/json' -d'
{
      "description":  "Smoothies are one of our favorite breakfast options year-round."
}
'
```

```bash
$ curl -X PUT "localhost:9200/cooking/_doc/2?pretty" -H 'Content-Type: application/json' -d'
{
      "description":  "A smoothie is a thick, cold beverage made from pureed raw fruit."
}
'
```

```bash
$ curl -X PUT "localhost:9200/cooking/_doc/3?pretty" -H 'Content-Type: application/json' -d'
{
      "description":  "Eggs Benedict is a traditional American breakfast or brunch dish."
}
'
```    
    
### Search the Index for Similar Documents
    
Let's look for documents that mention "breakfast":
    
```bash
$ curl -XGET 'localhost:9200/cooking/_search?q=description:breakfast&pretty'
```

And the response:

```bash
{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : 2,
    "max_score" : 0.48233607,
    "hits" : [
      {
        "_index" : "cooking",
        "_type" : "_doc",
        "_id" : "1",
        "_score" : 0.48233607,
        "_source" : {
          "description" : "Smoothies are one of our favorite breakfast options year-round."
        }
      },
      {
        "_index" : "cooking",
        "_type" : "_doc",
        "_id" : "3",
        "_score" : 0.48233607,
        "_source" : {
          "description" : "Eggs Benedict is a traditional American breakfast or brunch dish."
        }
      }
    ]
  }
}
```

But if we look for documents that mention "smoothie"...

```bash
$ curl -XGET 'localhost:9200/cooking/_search?q=description:smoothie&pretty'
```

We only get the second document back, since the word "smoothie" is pluralized in the first document:

```bash
{
  "took" : 1,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 0.9331132,
    "hits" : [
      {
        "_index" : "cooking",
        "_type" : "_doc",
        "_id" : "2",
        "_score" : 0.9331132,
        "_source" : {
          "description" : "A smoothie is a thick, cold beverage made from pureed raw fruit."
        }
      }
    ]
  }
}
```

We can work around this by using a fuzzy search, which will return both the first and second documents:

```bash
curl -XGET "localhost:9200/cooking/_search" -H 'Content-Type: application/json' -d'
{
    "query": {
       "fuzzy" : { "description" : "smoothie" }
    }
}
'
```
    

 
## Adding Documents in Bulk

Now let's try to add all of the cooking from the corpus to the index.

In [4]:
from elasticsearch.helpers import bulk
from elasticsearch import Elasticsearch

class ElasticIndexer(object):
    """
    Create an ElasticSearch instance, and given a list of documents, 
    index the documents into ElasticSearch.
    """
    def __init__(self, index_name, doc_type):
        self.type = doc_type
        self.index_name = index_name
        self.elastic_search = Elasticsearch()
        
    def make_documents(self, texts):        
        for document in texts:
            yield {
                "_index": "cooking",
                "_type": "_doc",
                "description": document
            }        
    def index(self, texts):
        bulk(self.elastic_search, self.make_documents(texts))

    def clear_elastic_search(self):
        self.elastic_search.indices.delete(
            index=self.index_name,
            ignore=[400, 404]
        )

In [5]:
indexer = ElasticIndexer("cooking", "_doc")
indexer.clear_elastic_search()
indexer.index(food_stories)

In [8]:
query = {"match_all": {}}
res = indexer.elastic_search.search(index="cooking", body={"query":query})

print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit)

Got 30 Hits:
{'_index': 'cooking', '_type': '_doc', '_id': 't9n5H2QB7PzBcXb-nYH5', '_score': 1.0, '_source': {'description': 'Tuesday, January 26, 2016 \n\n\n\n\nIf you didn’t have a nonna to do so when you were a \n\n, it’s more than likely that Marcella Hazan was the person who introduced you to the concept of a spaghetti frittata, a cozy mess of leftover spaghetti, scrambled egg, some butter, parsley and a fistful of parmesan, cooked in a skillet and cut into wedges. It’s unfancy food at its best, as should be no surprise from the woman who was very distressed by complicated chefs’ recipes, wondering \n\n\n\n\n\nSo when I first saw Food & Wine’s Cacio e Pepe Pasta Pie on Pinterest earlier this month, as one does, my first thought was “Oooh, so impossible-to-achieve outside a food styling studio pretty,” (because, I mean, look at it) followed by “Wait, that’s not cacio e pepe” (a Roman dish with exactly three ingredients — pecorino, black pepper and spaghetti, usually fresh tonnarell

In [9]:
query = {"fuzzy":{"description":"breakfast"}}
res = indexer.elastic_search.search(index="cooking", body={"query":query})
print("Got %d Hits:" % res['hits']['total'])
for hit in res['hits']['hits']:
    print(hit)

Got 8 Hits:
{'_index': 'cooking', '_type': '_doc', '_id': 'yNn5H2QB7PzBcXb-nYH5', '_score': 2.3950295, '_source': {'description': 'Ever wonder how your favorite chefs make their food taste and look so good? Tune in this weekend as Food Network chefs reveal secrets from their own kitchen, their time working at restaurants and more. First up on Saturday, Ree Drummond is answering her fans’ most-asked questions while she makes Mexican chicken soup and white sandwich bread from scratch. Then, the co-hosts of The Kitchen are spilling the secrets restaurants use to make your food taste more delicious, while making potato chip-crusted salmon and a pickle-brined pork chop.\n\n\n\nOn Sunday morning, Nancy Fuller is celebrating her farmhouse’s 250-year anniversary with a spread including beans-and-greens bruschetta and a bacon-wrapped pork roast. And on Sunday night, Guy Fieri is challenging four more chefs in the Grocery Games Redemption Tournament, but only one can move on to the finale to com